# FEATURE ENGINEERING

This notebook logically follows the preprocessing step and performs feature engineering for modeling or further analysis.

✔ What it does:

- Loads the cleaned data from the previous step (pre-processed.csv).

Creates new features:
- Price Change %
- Target (binary label: will price rise next day?)
- Date features: weekday, month, quarter
- Rolling averages (MA_5, MA_10)
- Daily Return and log of Adjusted Close price


In [1]:
import os
import pandas as pd
import numpy as np

file_path = "data\PREPROCESSING\merged_stock_income.csv" 

df_merged = pd.read_csv(file_path)

df_merged.info()

<>:5: SyntaxWarning: invalid escape sequence '\P'
<>:5: SyntaxWarning: invalid escape sequence '\P'
C:\Users\bburg\AppData\Local\Temp\ipykernel_20876\3320304327.py:5: SyntaxWarning: invalid escape sequence '\P'
  file_path = "data\PREPROCESSING\merged_stock_income.csv"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4502544 entries, 0 to 4502543
Data columns (total 32 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Shares Outstanding                        float64
 10  Currency                                  object 
 11  Fiscal Year                               float64
 12  Fiscal Period                             object 
 13  Publish Date                              object 
 14  Re

In [2]:
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common)
0,GOOG,2019-04-25,18,63.24,63.37,62.60,63.17,62.87,22145900,1.388488e+10,...,3.592800e+10,5.394000e+09,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10
1,HEES,2019-04-25,6767693,30.00,31.77,29.26,30.26,25.06,676844,3.578700e+07,...,2.113180e+08,-4.742400e+07,-54033000.0,1.638940e+08,1.683600e+07,1.807300e+08,-4.703600e+07,1.336940e+08,1.321700e+08,1.321700e+08
2,MANH,2019-04-25,105128,66.52,67.00,64.24,65.44,65.44,917088,6.459451e+07,...,1.159240e+08,1.530000e+05,715000.0,1.160770e+08,7.252000e+06,1.160770e+08,-3.031500e+07,8.576200e+07,8.576200e+07,8.576200e+07
3,ERII,2019-04-25,6767695,9.60,9.64,9.43,9.46,9.46,113678,5.411600e+07,...,1.036400e+07,1.892000e+06,2010000.0,1.225600e+07,-2.332000e+06,1.225600e+07,-1.343000e+06,1.091300e+07,1.091300e+07,1.091300e+07
4,CIM,2019-04-25,6767699,57.39,57.39,56.43,56.76,28.84,383803,6.236458e+07,...,5.052940e+08,9.390000e+05,2300000.0,5.052940e+08,1.692250e+08,6.745190e+08,-4.405000e+06,6.701140e+08,6.701140e+08,5.963500e+08


New Columns: 

✅ Price Change % → Added daily price change % calculation (1 day lag)

✅ Target → Created Target column for market movement prediction

✅ Weekday (Weekday) → Categorical feature (Monday-Sunday) for classification models.

✅ Month (Month) → Categorical month number (1-12) for classification models.

✅ Quarter (Quarter) → Categorical feature (1-4) for classification models.

✅ Daily Returns (Daily_Return) → Percentage change in adjusted close price, useful for financial modeling.

✅ Moving Averages (MA_5, MA_10) → 5-day and 10-day moving averages for trend detection.

✅ Log Returns (Log_Returns) → Helps in normalizing price data for time series models.


In [3]:
df_merged["Price Change %"] = df_merged["Close"].pct_change() * 100  # Use correct column name

df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %
0,GOOG,2019-04-25,18,63.24,63.37,62.60,63.17,62.87,22145900,1.388488e+10,...,5.394000e+09,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,NaN
1,HEES,2019-04-25,6767693,30.00,31.77,29.26,30.26,25.06,676844,3.578700e+07,...,-4.742400e+07,-54033000.0,1.638940e+08,1.683600e+07,1.807300e+08,-4.703600e+07,1.336940e+08,1.321700e+08,1.321700e+08,-52.097515
2,MANH,2019-04-25,105128,66.52,67.00,64.24,65.44,65.44,917088,6.459451e+07,...,1.530000e+05,715000.0,1.160770e+08,7.252000e+06,1.160770e+08,-3.031500e+07,8.576200e+07,8.576200e+07,8.576200e+07,116.259088
3,ERII,2019-04-25,6767695,9.60,9.64,9.43,9.46,9.46,113678,5.411600e+07,...,1.892000e+06,2010000.0,1.225600e+07,-2.332000e+06,1.225600e+07,-1.343000e+06,1.091300e+07,1.091300e+07,1.091300e+07,-85.544010
4,CIM,2019-04-25,6767699,57.39,57.39,56.43,56.76,28.84,383803,6.236458e+07,...,9.390000e+05,2300000.0,5.052940e+08,1.692250e+08,6.745190e+08,-4.405000e+06,6.701140e+08,6.701140e+08,5.963500e+08,500.000000


Create the Target Variable for ML
To predict whether the price will go up or down, shift the adjusted close price:

In [4]:
df_merged["Target"] = (df_merged["Close"].shift(-1) > df_merged["Close"]).astype(int)  # 1 if price rises, 0 if falls
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target
0,GOOG,2019-04-25,18,63.24,63.37,62.60,63.17,62.87,22145900,1.388488e+10,...,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,NaN,0
1,HEES,2019-04-25,6767693,30.00,31.77,29.26,30.26,25.06,676844,3.578700e+07,...,-54033000.0,1.638940e+08,1.683600e+07,1.807300e+08,-4.703600e+07,1.336940e+08,1.321700e+08,1.321700e+08,-52.097515,1
2,MANH,2019-04-25,105128,66.52,67.00,64.24,65.44,65.44,917088,6.459451e+07,...,715000.0,1.160770e+08,7.252000e+06,1.160770e+08,-3.031500e+07,8.576200e+07,8.576200e+07,8.576200e+07,116.259088,0
3,ERII,2019-04-25,6767695,9.60,9.64,9.43,9.46,9.46,113678,5.411600e+07,...,2010000.0,1.225600e+07,-2.332000e+06,1.225600e+07,-1.343000e+06,1.091300e+07,1.091300e+07,1.091300e+07,-85.544010,1
4,CIM,2019-04-25,6767699,57.39,57.39,56.43,56.76,28.84,383803,6.236458e+07,...,2300000.0,5.052940e+08,1.692250e+08,6.745190e+08,-4.405000e+06,6.701140e+08,6.701140e+08,5.963500e+08,500.000000,0


Handling Null Values: Since .pct_change() creates NaN values for the first row, fill them:

In [5]:
df_merged["Price Change %"].fillna(0, inplace=True)
df_merged["Target"].fillna(0, inplace=True)
df_merged.head()

C:\Users\bburg\AppData\Local\Temp\ipykernel_20876\287450061.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged["Price Change %"].fillna(0, inplace=True)
C:\Users\bburg\AppData\Local\Temp\ipykernel_20876\287450061.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target
0,GOOG,2019-04-25,18,63.24,63.37,62.60,63.17,62.87,22145900,1.388488e+10,...,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,0.000000,0
1,HEES,2019-04-25,6767693,30.00,31.77,29.26,30.26,25.06,676844,3.578700e+07,...,-54033000.0,1.638940e+08,1.683600e+07,1.807300e+08,-4.703600e+07,1.336940e+08,1.321700e+08,1.321700e+08,-52.097515,1
2,MANH,2019-04-25,105128,66.52,67.00,64.24,65.44,65.44,917088,6.459451e+07,...,715000.0,1.160770e+08,7.252000e+06,1.160770e+08,-3.031500e+07,8.576200e+07,8.576200e+07,8.576200e+07,116.259088,0
3,ERII,2019-04-25,6767695,9.60,9.64,9.43,9.46,9.46,113678,5.411600e+07,...,2010000.0,1.225600e+07,-2.332000e+06,1.225600e+07,-1.343000e+06,1.091300e+07,1.091300e+07,1.091300e+07,-85.544010,1
4,CIM,2019-04-25,6767699,57.39,57.39,56.43,56.76,28.84,383803,6.236458e+07,...,2300000.0,5.052940e+08,1.692250e+08,6.745190e+08,-4.405000e+06,6.701140e+08,6.701140e+08,5.963500e+08,500.000000,0


In [6]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# Show DataFrame info
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4502544 entries, 0 to 4502543
Data columns (total 34 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Shares Outstanding                        float64
 10  Currency                                  object 
 11  Fiscal Year                               float64
 12  Fiscal Period                             object 
 13  Publish Date                              object 
 14  Re

In [7]:
df_merged.columns

Index(['Ticker', 'Date', 'SimFinId', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Shares Outstanding', 'Currency', 'Fiscal Year',
       'Fiscal Period', 'Publish Date', 'Restated Date', 'Shares (Basic)',
       'Shares (Diluted)', 'Revenue', 'Cost of Revenue', 'Gross Profit',
       'Operating Expenses', 'Selling, General & Administrative',
       'Operating Income (Loss)', 'Non-Operating Income (Loss)',
       'Interest Expense, Net', 'Pretax Income (Loss), Adj.',
       'Abnormal Gains (Losses)', 'Pretax Income (Loss)',
       'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations', 'Net Income',
       'Net Income (Common)', 'Price Change %', 'Target'],
      dtype='object')

Weekday column

In [8]:
df_merged['Date'] = pd.to_datetime(df_merged['Date']) # converting the "Date" column to a Datetime Object (double checking)

In [9]:
# 1. Weekday (Categorical for Classification)
df_merged['Weekday'] = df_merged['Date'].dt.day_name()  # 'Monday', 'Tuesday', etc.

Month Column

In [10]:
# 2. Month (Categorical for Classification)
df_merged['Month'] = df_merged['Date'].dt.month  # 1 (Jan) to 12 (Dec)

Quarter Column

In [11]:
# 3. Quarter (Categorical for Classification)
df_merged['Quarter'] = df_merged['Date'].dt.quarter  # 1 to 4

Daily Returns (Percentage Change)

In [12]:
df_merged['Daily_Return'] = df_merged['Close'].pct_change() * 100  # Convert to %

Moving Averages (5-day and 10-day)

In [13]:
df_merged['MA_5'] = df_merged['Close'].rolling(window=5, min_periods=1).mean()
df_merged['MA_10'] = df_merged['Close'].rolling(window=10, min_periods=1).mean()

Log Returns for Time Series Modeling

In [14]:
# Compute daily log-returns
df_merged['Log_Returns'] = np.log(df_merged['Close'] / df_merged['Close'].shift(1))

c:\Users\bburg\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
df_merged.set_index('Date', inplace=True)

In [16]:
df_merged.head()

,Ticker,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,Currency,Fiscal Year,Fiscal Period,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target,Weekday,Month,Quarter,Daily_Return,MA_5,MA_10,Log_Returns
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-25,GOOG,18,63.24,63.37,62.60,63.17,62.87,22145900,1.388488e+10,USD,2019.0,FY,2020-02-04,2022-02-02,1.385192e+10,1.490166e+10,1.618570e+11,-7.189600e+10,8.996100e+10,-5.403300e+10,-2.801500e+10,3.592800e+10,5.394000e+09,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,0.000000,0,Thursday,4,2,NaN,63.170000,63.170000,NaN
2019-04-25,HEES,6767693,30.00,31.77,29.26,30.26,25.06,676844,3.578700e+07,USD,2022.0,FY,2023-02-22,2025-02-21,3.594300e+07,3.608900e+07,1.244518e+09,-6.893550e+08,5.551630e+08,-3.438450e+08,-3.438450e+08,2.113180e+08,-4.742400e+07,-54033000.0,1.638940e+08,1.683600e+07,1.807300e+08,-4.703600e+07,1.336940e+08,1.321700e+08,1.321700e+08,-52.097515,1,Thursday,4,2,-52.097515,46.715000,46.715000,-0.736003
2019-04-25,MANH,105128,66.52,67.00,64.24,65.44,65.44,917088,6.459451e+07,USD,2019.0,FY,2020-02-10,2022-02-07,6.439700e+07,6.510300e+07,6.179490e+08,-2.849670e+08,3.329820e+08,-2.170580e+08,-1.214630e+08,1.159240e+08,1.530000e+05,715000.0,1.160770e+08,7.252000e+06,1.160770e+08,-3.031500e+07,8.576200e+07,8.576200e+07,8.576200e+07,116.259088,0,Thursday,4,2,116.259088,52.956667,52.956667,0.771307
2019-04-25,ERII,6767695,9.60,9.64,9.43,9.46,9.46,113678,5.411600e+07,USD,2019.0,FY,2020-03-06,2022-02-24,5.474000e+07,5.606700e+07,8.694200e+07,-2.033500e+07,6.660700e+07,-5.624300e+07,-3.226600e+07,1.036400e+07,1.892000e+06,2010000.0,1.225600e+07,-2.332000e+06,1.225600e+07,-1.343000e+06,1.091300e+07,1.091300e+07,1.091300e+07,-85.544010,1,Thursday,4,2,-85.544010,42.082500,42.082500,-1.934061
2019-04-25,CIM,6767699,57.39,57.39,56.43,56.76,28.84,383803,6.236458e+07,USD,2021.0,FY,2022-02-17,2024-02-29,7.791570e+07,8.182412e+07,9.375460e+08,-3.266280e+08,6.109180e+08,-1.056240e+08,-2.224600e+07,5.052940e+08,9.390000e+05,2300000.0,5.052940e+08,1.692250e+08,6.745190e+08,-4.405000e+06,6.701140e+08,6.701140e+08,5.963500e+08,500.000000,0,Thursday,4,2,500.000000,45.018000,45.018000,1.791759


In [18]:
df_merged['Log_Returns'].dropna(inplace=True)

In [19]:
df_merged['Log_Returns'].describe()

c:\Users\bburg\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


count    4.502517e+06
mean              NaN
std               NaN
min              -inf
25%     -1.417481e+00
50%      0.000000e+00
75%      1.417047e+00
max               inf
Name: Log_Returns, dtype: float64

In [20]:
# Replace inf/-inf with NaN
df_merged['Log_Returns'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop missing values
df_merged.dropna(subset=['Log_Returns'], inplace=True)

C:\Users\bburg\AppData\Local\Temp\ipykernel_20876\3151199279.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['Log_Returns'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [21]:
df_merged['Log_Returns'].describe()

count    4.483847e+06
mean    -6.131044e-05
std      2.295275e+00
min     -1.798928e+01
25%     -1.410339e+00
50%      0.000000e+00
75%      1.409907e+00
max      1.866754e+01
Name: Log_Returns, dtype: float64

In [22]:
df_merged['Close'].isna().sum()
df_merged['Close'].eq(0).sum()

0

In [23]:
# Save enriched dataset
os.makedirs("data/ENRICH", exist_ok=True)
df_merged.to_csv("data/ENRICH/merged_stock_income.csv", index=True) 